In [1]:
import sys
sys.path.append('../')

from loguru import logger as log

In [2]:
import numpy as np
from ortools.sat.python import cp_model

from vrp_study.data_model import Tariff, Cargo, Node
from vrp_study.data_model import TariffCost, Route
from vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from vrp_study.pdptw_model.routing_model import find_optimal_paths

In [3]:
benchmark_type = 'pdp_400'
name = 'LC1_4_9.txt'

In [4]:
from typing import Optional

tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [5]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


In [6]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1501, service_time=0, coordinates=(100, 100))

In [7]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [8]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [9]:
from vrp_study.configs import ModelConfig

routing_manager_builder = PDRoutingManagerBuilder(
    distance_matrix = distance_matrix,
    time_matrix = time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager_builder.add_cargos(cargos)
routing_manager_builder.add_tariff(tariff)

routing_manager_builder.add_depo(depo)

# routing_manager.distance_matrix = distance_matrix
# routing_manager.time_matrix = time_matrix


routing_manager = routing_manager_builder.build()

In [10]:
from vrp_study.pdptw_model.solution_builder import  SolutionBuilder

# sol = find_optimal_paths(routing_manager, SolutionBuilder())

In [15]:
from vrp_study.pdptw_model.solution_builder import SolutionBuilder
from vrp_study.pdptw_model.routing_model import get_optimal_model_params, PDP_CONSTRAINTS, get_solution, SolutionCallback
from vrp_study.solver import VehicleRoutingSolver


vrp_solver = VehicleRoutingSolver(routing_manager=routing_manager,
                                  solution_callback=SolutionCallback,
                                  solution_processing=None,
                                  search_parameters=get_optimal_model_params(), 
                                  initial_solution_builder=None, 
                                  constraints=PDP_CONSTRAINTS)

vrp_solver.solve()

2025-07-07 15:27:26.556 | INFO     | vrp_study.solver:solve:166 - problem size: 419
2025-07-07 15:27:26.559 | INFO     | vrp_study.solver:solve_routing:101 - Начало создания модели
2025-07-07 15:27:26.575 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:304 - Добавление размерности для расстояния
2025-07-07 15:27:26.576 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:341 - Добавление размерности для расстояния
2025-07-07 15:27:26.577 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:273 - Добавление ограничения для порядка доставки
2025-07-07 15:27:26.579 | INFO     | vrp_study.pdptw_model.routing_model:add_vehicles_cost:246 - Добавление стоимостей машин
2025-07-07 15:27:26.584 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:195 - add time
2025-07-07 15:27:26.588 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-07-07 15:27:26.594 | INFO   

(799014,
 {0: Route(id=0, path=[0, 147, 148, 19, 315, 316, 285, 286, 20, 253, 254, 0], tariff=Tariff(id='car', capacity=200, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=10000, cost_per_km=1, fixed_cost=0)], max_count=100)),
  1: Route(id=1, path=[0, 297, 298, 81, 323, 324, 287, 311, 312, 288, 203, 204, 82, 0], tariff=Tariff(id='car', capacity=200, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=10000, cost_per_km=1, fixed_cost=0)], max_count=100)),
  2: Route(id=2, path=[0, 95, 99, 96, 100, 1, 197, 215, 198, 2, 216, 0], tariff=Tariff(id='car', capacity=200, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=10000, cost_per_km=1, fixed_cost=0)], max_count=100)),
  3: Route(id=3, path=[0, 277, 173, 174, 5, 278, 6, 0], tariff=Tariff(id='car', capacity=200, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=10000, cost_per_km=1, fixed_cost=0)], max_count=100)),
  4: Route(id=4, path=[0, 73, 39, 301, 61, 62, 91, 92, 302, 74, 40, 0], tariff=Tariff(id='car', capacity

In [12]:
#  best_score: 107
# 2025-06-02 14:45:05,380 - routing_model [INFO] - best_score: 46228.27

In [13]:
# est_score: 8
# 2025-06-02 14:25:30,179 - routing_model [INFO] - best_score: 1979.21

In [14]:
total_l = 0
for s in sol[0]:
    if len(s) > 0:
        # print(s)
        # a = routing_manager.nodes()[s[i]]
        # b = routing_manager.nodes()[s[i+1]]
        
        l = sum(routing_manager.get_distance(routing_manager.nodes()[s[i]], routing_manager.nodes()[s[i + 1]]) for i in range(len(s) - 1))
        # print(s, l)
        total_l += l
total_l

NameError: name 'sol' is not defined

In [ ]:
type(routing_manager)

vrp_study.routing_manager.RoutingManager

In [ ]:
for crg in cargos:
    in1 = routing_manager_builder._node_to_inner_node[crg.nodes[0]]
    in2 = routing_manager_builder._node_to_inner_node[crg.nodes[1]]

    start = None
    end = None
    in_time = None
    out_time = None

    for i, s in enumerate(sol[0]):
        s_time = sol[1][i]
        if len(s) == 0:
            continue

        for order, i in enumerate(s):
            if i == in1.id:
                start = i
                in_time = s_time[order]
            if i == in2.id:
                end = i
                out_time = s_time[order]

    assert start is not None and end is not None and start < end, (start, end)

    if not (crg.nodes[0].start_time <= in_time[0] and crg.nodes[0].end_time >= in_time[1]):
        print(in_time, crg.nodes[0].start_time, crg.nodes[0].end_time)
    if not (crg.nodes[1].start_time <= out_time[0] and crg.nodes[1].end_time >= out_time[1]):
        print(out_time, crg.nodes[0].start_time, crg.nodes[0].end_time)